# Import data (XMP, EXIF and Images) to Pandas

In [ ]:
import struct, base64, zlib, binascii, CppHeaderParser, re, collections, logging, sys, itertools, glob, json, \
    pyexiv2, fractions, os, numpy, pandas
from wand.image import Image
from wand.display import display
try:
    from cStringIO import StringIO
except ImportError:
    from StringIO import StringIO

In [ ]:
logger = logging.getLogger()
log_formatter = logging.Formatter('%(asctime)-15s %(levelname)10s %(message)s')
stderr_handler = logging.StreamHandler(sys.stderr)
stderr_handler.setFormatter(log_formatter)
stderr_handler.setLevel(logging.DEBUG)
file_handler = logging.FileHandler('import.log')
file_handler.setFormatter(log_formatter)
file_handler.setLevel(logging.DEBUG)
logger.handlers = []
logger.addHandler(stderr_handler)
logger.addHandler(file_handler)
logger.setLevel(logging.DEBUG)

## Detect data structures that we need to parse

In [ ]:
STRUCTS = {
           'dt_iop_tonecurve_node_t' : 'ff',
           'dt_iop_vector_2d_t' : 'ff'
           }
ENUMS = frozenset({
                   'dt_image_orientation_t',
                   'dt_gaussian_order_t',
                   'dt_iop_shadhi_algo_t',
                   'dt_iop_lowpass_algo_t',
                   'dt_colorspaces_color_profile_type_t',
                   'dt_iop_colormapping_flags_t',
                   'dt_iop_ashift_mode_t',
                   'dt_iop_ashift_crop_t',
                   'dt_iop_tonecurve_node_t',
                   'dt_iop_colortransfer_flag_t',
                   'dt_colorspaces_color_mode_t',
                   'dt_iop_dither_type_t',
                   'dt_iop_watermark_base_scale_t',
                   'dt_iop_defringe_mode_t',
                   'dt_iop_colorzones_channel_t',
                   'dt_iop_denoiseprofile_mode_t',
                   'dt_iop_highlights_mode_t',
                   '_dt_iop_grain_channel_t',
                   'dt_iop_levels_mode_t'
                   })
ENUM_FMT = 'I'
DEFAULT_FMT = 'I'
TYPE_RE = (
    (re.compile(r'unsigned char|uchar|uint8'), 'B'),
    (re.compile(r'char|int8'), 'c'),
    (re.compile(r'bool'), '?'),
    (re.compile(r'unsigned short|ushort|uint16'), 'H'),
    (re.compile(r'short|int16'), 'h'),
    (re.compile(r'uint32'), 'I'),
    (re.compile(r'int32'), 'i'),
    (re.compile(r'unsigned long long|uint64'), 'Q'),
    (re.compile(r'long long|int64'), 'q'),
    (re.compile(r'unsigned long|ulong|uint'), 'Q'),
    (re.compile(r'long|int'), 'q'),
    (re.compile(r'float'), 'f'),
    (re.compile(r'double'), 'd'),
    (re.compile(r'char\[]'), 's')
)
CONSTANTS = {
             'DT_IOP_COLOR_ICC_LEN' : 100,
             'HISTN' : 1 << 11,
             'MAXN' : 5,
             'CHANNEL_SIZE' : 7,
             'MAX_ZONE_SYSTEM_SIZE' : 24,
             'DT_IOP_LOWLIGHT_LUT_RES' : 0x10000,
             'atrous_none' : 5
             }
ARRAY_SIZES = {
               'dt_iop_zonesystem_params_t' : { 'zone' : 25 },
               'dt_iop_colorchecker_data_t' : {
                                               'source_Lab' : 3 * 49,
                                               'coeff_L' : 49 + 4,
                                               'coeff_a' : 49 + 4,
                                               'coeff_b' : 49 + 4
                                               },
               'dt_iop_colorbalance_data_t' : { 'lift' : CONSTANTS['CHANNEL_SIZE'] }
}
def build_typedef(c_def_string, align = '='):
    parsed = CppHeaderParser.CppHeader(c_def_string, argType = 'string')
    definition = next(iter(parsed.classes.values()))
    fmt = align
    names = []
    sizes = []
    for field in definition['properties']['public']:
        names.append(field['name'])

        fmt_factor = 1
        if int(field['array']):
            known_size = ARRAY_SIZES.get(definition['name'], {}).get(field['name'], None)
            if not known_size is None:
                fmt_factor = known_size
            else:
                if 'array_size' in field:
                    try:
                        if isinstance(field['array_size'], str):
                            fmt_factor = int(field['array_size'], 0)
                        else:
                            fmt_factor = int(field['array_size'])
                    except ValueError:
                        if field['array_size'] in CONSTANTS:
                            fmt_factor = CONSTANTS[field['array_size']]
                        else:
                            logger.warning('%s parse error: %s is array, could not parse size %s' % (definition['name'],
                                                                                                     field['name'],
                                                                                                     field['array_size']))
                else:
                    logger.warning('%s parse error: %s is array, but size is unknown' % (definition['name'],
                                                                                         field['name']))
        sizes.append(fmt_factor)

        field_fmt = None
        ftype = field['type']
        if ftype in STRUCTS:
            field_fmt = STRUCTS[ftype]
        elif ftype in ENUMS:
            field_fmt = ENUM_FMT
        else:
            for regex, t in TYPE_RE:
                if regex.search(ftype):
                    field_fmt = t
                    break
        if field_fmt is None:
            logger.warning('%s parse error: could not determine format of field %s (type %s)' % (definition['name'],
                                                                                                 field['name'],
                                                                                                 ftype))
            field_fmt = DEFAULT_FMT
        fmt += field_fmt*fmt_factor
    return (fmt, names, sizes)


STRUCT_SUFFIXES = ('data', 'params')
def try_extract_filter_info(filename, filter_name = None):
    logger.info('Extracting filter info from %s' % filename)
    if filter_name is None:
        filter_name = os.path.splitext(os.path.basename(filename))[0]
    with open(filename, 'r') as f:
        content = f.read()
    for suffix in STRUCT_SUFFIXES:
        data_def = re.search(r'typedef struct dt_iop_%s_%s_t.*?dt_iop_%s_%s_t;' % (filter_name,
                                                                                   suffix,
                                                                                   filter_name,
                                                                                   suffix),
                             content,
                             re.DOTALL)
        if data_def:
            return filter_name, build_typedef(data_def.group(0))
    return None

def build_filter_info_table(darktable_src_dir):
    plugins_dir = os.path.join(darktable_src_dir, 'src', 'iop')
    return { info[0] : info[1]
            for info
            in (try_extract_filter_info(os.path.join(plugins_dir, f))
                for f
                in os.listdir(plugins_dir))
            if not info is None
           }

In [ ]:
# filters_info = build_filter_info_table('/home/windj/projects/thirdparty/darktable')
# with open('data/filters.js', 'w') as f:
#     json.dump(filters_info, f, indent = 4)

In [ ]:
def parse_struct(data, fmt, names, sizes):
    if data.startswith('gz'):
        data = data[4:]
        data = base64.decodestring(data)
        data = zlib.decompress(data)
    else:
        data = binascii.unhexlify(data)
    needed_size = struct.calcsize(fmt)
    if len(data) != needed_size:
        logger.error('parse_struct: %d bytes expected, %d bytes got' % (needed_size, len(data)))
        return None
    data = struct.unpack(fmt, data)
    result = {}
    i = 0
    for name, size in itertools.izip(names, sizes):
        if size == 1:
            result[name] = data[i]
        else:
            result[name] = data[i:i + size]
        i += size
    return result

class StructParser(object):
    def __init__(self, filter_info):
        self.filter_info = filter_info

    def __call__(self, data):
        return parse_struct(data, *self.filter_info)


class JointParser(object):
    def __init__(self, filters_info):
        self.parsers = { name : StructParser(info)
                        for name, info
                        in filters_info.viewitems() }
    
    def __call__(self, filter_name_data_pairs):
        result = {}
        for name, data in filter_name_data_pairs:
            parser = self.parsers.get(name)
            result['%s.ENABLED' % name] = 1
            if parser is None:
                logger.warning('Unknown filter %s!' % name)
                continue
            parse_res = parser(data)
            if parse_res is None:
                logger.warning('Could not parse %s data %s' % (name, data))
                continue
            for k, v in parse_res.viewitems():
                result['%s.%s' % (name, k)] = v
        return result

## Parse metadata

In [ ]:
def get_xmp_data(filename, filter_data_parser):
    metadata = pyexiv2.ImageMetadata(filename)
    metadata.read()
    filters_data = ((name, data)
                    for name, data, enabled
                    in itertools.izip(metadata['Xmp.darktable.history_operation'].value,
                                      metadata['Xmp.darktable.history_params'].value,
                                      metadata['Xmp.darktable.history_enabled'].value)
                    if enabled)
    result = {}
    try:
        result['rating'] = metadata['Exif.Image.Rating'].value
    except KeyError:
        pass
    result.update(filter_data_parser(filters_data))
    return result

IGNORED_IMAGE_METADATA = frozenset({
                                    'Exif.Canon.AFInfo',
                                    'Exif.Canon.CameraInfo',
                                    'Exif.Photo.MakerNote',
                                    'Exif.Canon.DustRemovalData',
                                    'Exif.Canon.0x0098',
                                    'Exif.Canon.CustomFunctions',
                                    'Exif.Canon.SensorInfo',
                                    'Exif.Canon.ColorData',
                                    'Exif.Canon.0x4002',
                                    'Exif.Canon.0x4005',
                                    'Exif.Canon.0x4008',
                                    'Exif.Canon.0x4009',
                                    'Exif.Canon.0x4010',
                                    'Exif.Canon.0x4011',
                                    'Exif.Canon.0x4012',
                                    'Exif.Canon.0x4015',
                                    'Exif.Canon.0x4016',
                                    'Exif.Canon.0x4017',
                                    'Exif.Canon.0x4018',
                                    'Exif.Canon.0x4019',
                                    'Exif.Photo.UserComment'
                                    })

def is_useless_str(val):
    if not isinstance(val, str):
        return False
    try:
        int(val)
        return True # int as string is useless
    except ValueError:
        pass
    return len(val) > 10

def convert_value(val):
    if isinstance(val, fractions.Fraction):
        return float(val)
    return val

def get_image_metadata(filename):
    metadata = pyexiv2.ImageMetadata(filename)
    metadata.read()
    return { k : convert_value(t.value)
            for k, t
            in metadata.items()
            if not k in IGNORED_IMAGE_METADATA and not is_useless_str(t.value) }

## Get graphical content

In [ ]:
def get_image_content(filename, max_size = 1000):
    with open(filename, 'r+') as f:
        ext = os.path.splitext(os.path.basename(filename))[1][1:]
        with Image(blob = f.read(), format = ext) as img:
            vertical = img.size[0] < img.size[1]
            if vertical:
                width = img.size[0] * max_size / img.size[1]
                height = max_size
                need_rotate = True
            else:
                width = max_size
                height = img.size[0] * max_size / img.size[1]
                need_rotate = False
            img.resize(int(width), int(height))
            if vertical:
                img.rotate(90)
            blob = img.make_blob(format='RGB')
            pixels = numpy.zeros((width, height, 3))
            for col in range(width):
                for row in range(height):
                    pixel_i = (row * width + col) * 3
                    for channel in xrange(3):
                        pixels[col, row, channel] = ord(blob[pixel_i + channel]) / 256.0
            return pixels

## Join all data sources and save data to Pandas

In [ ]:
ALLOWED_IMAGE_EXTENSIONS = ('CR2')
OUTCOMES_PREFIX = 'OUTCOMES'
META_PREFIX = 'META'
IMAGE_CONTENT_FIELD = 'CONTENT'
def build_dataframe(in_directory, xmp_parser, max_size = 1000):
    result = []
    indices = []
    for xmp_file in os.listdir(in_directory):
        xmp_file = os.path.join(in_directory, xmp_file)
        img_file, ext = os.path.splitext(xmp_file)
        ext = ext.lower()
        if ext != '.xmp' or not os.path.isfile(xmp_file) or not os.path.isfile(img_file):
            logger.debug('Skipping %s' % xmp_file)
            continue
        logger.debug('Processing %s' % xmp_file)
        img_res = { '.'.join((OUTCOMES_PREFIX, n)) : v
                   for n, v
                   in get_xmp_data(xmp_file, xmp_parser).viewitems() }
        img_res.update(('.'.join((META_PREFIX, n)), v)
                       for n, v
                       in get_image_metadata(img_file).viewitems())
        img_res[IMAGE_CONTENT_FIELD] = get_image_content(img_file)
        result.append(img_res)
        indices.append(img_file)
    return pandas.DataFrame(data = result, index = indices)

In [ ]:
with open('data/filters.js', 'r') as f:
    final_filters_info = json.load(f)
final_filters_parser = JointParser(final_filters_info)

In [ ]:
build_dataframe('samples/', final_filters_parser)